# Atividade sobre métricas de efetividade

O objetivo desta atividade é entender melhor as métricas de efetividade apresentadas durante a aula.

Para realizar esta atividade primeiramente é necessário treinar um modelo e computar sua matriz de confusão. O modelo treinado será bastante simples e não serão feitas grandes extrações de características.

O dataset utilizado será de históricos de incêndios no Brasil, presentes [neste link](https://www.kaggle.com/gustavomodelli/forest-fires-in-brazil).

In [1]:
import pandas as pd

incendios_br = pd.read_csv("https://orionwinter.github.io/datasets/forest_fires_in_brazil.csv")

incendios_br.head()

,year,state,month,number
0,1998,Acre,1,0.0
1,1999,Acre,1,0.0
2,2000,Acre,1,0.0
3,2001,Acre,1,0.0
4,2002,Acre,1,0.0


Como a variável resposta deve ser categórica para calcularmos as métricas de classificação, vamos transformar do número de incêndios para um booleano que representa se o número de incêncios foi maior que a mediana do estado.

In [2]:
mediana_estado = incendios_br.groupby(
    "state"
).agg(
    {"number": "median"}
).rename(
    columns={"number": "mediana_incendios"}
)

mediana_estado.head()

,mediana_incendios
state,
Acre,2.188
Alagoas,10.000
Amapa,2.000
Amazonas,23.000
Bahia,140.000


In [3]:
incendios_br_proc = incendios_br.merge(
    mediana_estado, on="state"
).assign(
    acima_mediana = lambda row: row.number > row.mediana_incendios
).drop(
    columns=["mediana_incendios", "number"]
)

incendios_br_proc.head()

,year,state,month,acima_mediana
0,1998,Acre,1,False
1,1999,Acre,1,False
2,2000,Acre,1,False
3,2001,Acre,1,False
4,2002,Acre,1,False


Como os modelos não sabem lhe dar com variáveis categóricas, vamos transformá-las em numéricas, através de dummy variables.

In [4]:
incendios_br_proc = pd.get_dummies(incendios_br_proc, prefix=['state'])

incendios_br_proc

,year,month,acima_mediana,state_Acre,state_Alagoas,state_Amapa,state_Amazonas,state_Bahia,state_Ceara,state_Distrito Federal,state_Espirito Santo,state_Goias,state_Maranhao,state_Mato Grosso,state_Minas Gerais,state_Para,state_Paraiba,state_Pernambuco,state_Piau,state_Rio,state_Rondonia,state_Roraima,state_Santa Catarina,state_Sao Paulo,state_Sergipe,state_Tocantins
0,1998,1,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1999,1,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2000,1,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2001,1,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2002,1,False,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6449,2012,12,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6450,2013,12,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6451,2014,12,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6452,2015,12,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


Precisamos também separar em treino e teste, utilizando a proporção 80%/20% respectivamente.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    incendios_br_proc.drop(columns="acima_mediana"), 
    incendios_br_proc.acima_mediana, 
    test_size=0.2, 
    random_state=42
)

Agora vamos treinar o modelo utilizando 5 vizinhos mais próximos.

In [0]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(5)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

Por último, vamos gerar a matriz de confusão:

In [7]:
from sklearn.metrics import confusion_matrix

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
(tn, fp, fn, tp)

(490, 199, 148, 454)

## Agora é sua vez

Calcule as métricas aprendidas durante a aula: **acurácia, recall, precisão, f-measure, informedness e markedness**. Em seguida responda se o modelo teve boa eficiência e por quê.

In [15]:
#Acurácia
accuracy = (tp+tn)/(tp+tn+fn+fp)
print("Accuracy:",accuracy)
#Recall: given a positive example, how certain are we that S will identify the example as positive.
recall = tp/(tp+fn)
print("Recall:",recall)
#Precisão: when S makes a positive prediction, how certain are we the prediction is correct.
precision = tp/(tp+fp)
print("Precision:",precision)
#F-measure
fmeasure = 2 * ((precision * recall)/(precision + recall))
print("F-measure:", fmeasure)
#Informedness: a measure of how informed system M is about positives and negatives. is counterpart of recall
rp = tp+fn
rn = tn+fp
informedness = (tp/rp)-(fp/rn)
print("Informedness:",informedness)
#Markedness: a measure of trustworthiness of positive and negative predictions by system M. is the counterpart of precision
pp = tp+fp
pn = fn+tn
markedness = (tp/pp)-(fn/pn)
print("Markedness:",markedness)

Accuracy: 0.7312161115414407
Recall: 0.7541528239202658
Precision: 0.6952526799387443
F-measure: 0.7235059760956175
Informedness: 0.46532844075626
Markedness: 0.46327775830865026


Considerando as metricas informedness e markedness, vemos que o valor encontrado é de +0.46, o que significa que modelo mais falha do erra ao classificar os positivos como positivos. Caso o resultado fosse negativo, significaria que ele mais erra do que acerta. 1 significa um modelo totalmente perfeito e -1 totalmente perfeito em errar. 0 é que o modelo não consegue fazer nada. É interessante notar que os valores de recall, precisão e f-measure ficam por volta de 70%, mas isso acontece porque eles só consideram os valores positivos para a análise. A acuracia de 73% indica que dados todos os valores, a taxa de acerto é alta, mas ela não consegue medir sozinha a efetividade do modelo.

Sendo assim, podemos dizer esse modelo é não é dos melhores, pois ele não é bom nem para idenficar os casos verdadeiros (1) nem o contrario (-1).